In [13]:
import json
import datasets
import pandas as pd
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm
from datasets import load_dataset

In [2]:
dataset = load_dataset('alsubari/Israel-palestine-war')

Found cached dataset parquet (C:/Users/harri/.cache/huggingface/datasets/alsubari___parquet/alsubari--Israel-palestine-war-bcbb3b19f11f72aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'publish_channel', 'title', 'start_duration', 'end_duration', 'lengthSeconds', 'total_comments', 'total_unique_users', 'comments'],
        num_rows: 310
    })
})


In [14]:
metadata = []
for d in tqdm(dataset['train']):
    md = {}
    for k in d:
        if k != "comments":
            md[k] = d[k]
    metadata.append(md)

df_metadata = pd.DataFrame(metadata)

  0%|          | 0/310 [00:00<?, ?it/s]

In [16]:
comments_data = []
for d in tqdm(dataset['train']):
    for k in d:
        if k == "comments":
            for c in d[k]:
                c['video_id'] = d['id']
                comments_data.append(c)

df_comments = pd.DataFrame(comments_data)

  0%|          | 0/310 [00:00<?, ?it/s]

In [17]:
df_metadata.head()

,id,publish_channel,title,start_duration,end_duration,lengthSeconds,total_comments,total_unique_users
0,FUzLx1l37RE,NBCNews,Blinken reiterates Israel support but urges re...,2023-11-03 19:38:34,2023-11-04 17:38:34,241,25,21
1,PcQaG4sC9BM,NBCNews,Full Special Report: Israel declares war after...,2023-10-07 19:42:03,2023-11-04 12:38:36,1484,21905,14860
2,X2t6NyGP1-w,NBCNews,Hamas official claims it's their 'legal right'...,2023-11-03 19:49:36,2023-11-04 19:46:34,335,1853,1170
3,cd24N0tF-jY,NBCNews,FBI director warns of growing domestic threat ...,2023-11-01 19:50:24,2023-11-04 18:50:41,143,987,670
4,ICR5ml2YPkI,NBCNews,Israel’s president: 23-year-old Israeli hostag...,2023-10-31 19:51:23,2023-11-04 19:44:35,162,5275,3475


In [22]:
len(df_metadata.id.unique())

310

In [23]:
len(df_comments.author.unique())

457373

In [24]:
df_metadata.to_csv('data/alsubari_videos_metadata.csv')

In [25]:
df_comments.to_csv('data/alsubari_videos_comments.csv')